In [32]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data.head()


,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


In [66]:
from sklearn.preprocessing import StandardScaler


X = train_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]

y= train_data[['rainfall']] 

X_train, X_test, y_train, y_test  = train_test_split(X,y, random_state=0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LogisticRegression()
model2 = LinearRegression()
model.fit(X_train, y_train)
model2.fit(X_train, y_train)

prediction  = model.predict(X_test)
prediction2 = model2.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))
print('Accuracy of linear regression classifier on test set: {:.2f}'.format(model2.score(X_test, y_test)))

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of logistic regression classifier on test set: 0.87
Accuracy of linear regression classifier on test set: 0.46


c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [72]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model
log_reg = LogisticRegression(max_iter=2000)

# Define the parameter grid (Make sure 'l1' is only used with solvers that support it)
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],          # Regularization strength
    'penalty': ['l1', 'l2'],               # Regularization type
    'solver': ['liblinear', 'lbfgs', 'saga']  # Solvers
}

# Adjust the parameter grid to only use 'l1' with 'liblinear' and 'saga'
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],  # 'l2' works with 'lbfgs', 'liblinear', and 'saga'
    'solver': ['liblinear', 'lbfgs', 'saga']
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Fit the grid search to your data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and the best score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_log_reg = grid_search.best_estimator_
print("Test Set Accuracy:", best_log_reg.score(X_test_scaled, y_test))

test_data.fillna(0, inplace=True)
test = test_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]
test = scaler.transform(test)
print(test)

prediction = best_log_reg.predict(test)
result = pd.DataFrame({'id': test_data['id'], 'prediction': prediction})
result.to_csv('result.csv', index=False)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Accuracy: 0.8648083623693379
Test Set Accuracy: 0.864963503649635
[[ 1.04920011 -1.56259142 -1.56223564 ... -1.02289282 -0.70960487
   0.24407771]
 [ 0.52308344 -1.56259142 -1.42850259 ... -1.02289282 -0.70960487
   1.34593514]
 [ 1.8208379  -2.67166284 -2.59389058 ... -1.02289282 -0.83371069
  -0.49717184]
 ...
 [ 1.61039123 -0.94644063 -1.04640816 ... -1.02289282 -0.70960487
   0.00367245]
 [ 0.15480177 -0.94644063 -0.75983735 ... -1.02289282 -0.70960487
   1.76664435]
 [ 1.29472122 -0.73518893 -0.989094   ... -0.96770669 -0.58549905
   2.87851867]]


c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
from sklearn.svm import LinearSVC

clf = LinearSVC().fit(X_train, y_train)
print('Accuracy of Linear SVC classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Linear SVC classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))
test_data.fillna(0, inplace=True)     
test = test_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]
predictionSVC = clf.predict(test)  
result = pd.DataFrame({'id': test_data['id'], 'prediction': predictionSVC})
result.to_csv('resultSVC.csv', index=False)     

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Linear SVC classifier on training set: 0.86
Accuracy of Linear SVC classifier on test set: 0.87


In [35]:
test_data.fillna(0, inplace=True)
test = test_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]
print(test)

prediction = model.predict(test)
print(prediction) 
result = pd.DataFrame({'id': test_data['id'], 'prediction': prediction})
result.to_csv('result.csv', index=False)  

     pressure  maxtemp  temparature  mintemp  dewpoint  humidity  cloud  \
0      1019.5     17.5         15.8     12.7      14.9      96.0   99.0   
1      1016.5     17.5         16.5     15.8      15.1      97.0   99.0   
2      1023.9     11.2         10.4      9.4       8.9      86.0   96.0   
3      1022.9     20.6         17.3     15.2       9.5      75.0   45.0   
4      1022.2     16.1         13.8      6.4       4.3      68.0   49.0   
..        ...      ...          ...      ...       ...       ...    ...   
725    1020.8     18.2         17.6     16.1      13.7      96.0   95.0   
726    1011.7     23.2         18.1     16.0      16.0      78.0   80.0   
727    1022.7     21.0         18.5     17.0      15.5      92.0   96.0   
728    1014.4     21.0         20.0     19.7      19.8      94.0   93.0   
729    1020.9     22.2         18.8     17.0      13.3      79.0   89.0   

     sunshine  winddirection  windspeed  
0         0.0           50.0       24.3  
1         0.0  

In [41]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=0)
xgb_model.fit(X_train, y_train)

from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(objective='reg:squarederror')

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.9, 1.0]
}

grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best R² score:", grid_search.best_score_)

print("XGBoost R²:", xgb_model.score(X_test, y_test)) 

Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.7}
Best R² score: 0.445162832736969
XGBoost R²: 0.4032767415046692


In [43]:
xgb = XGBRegressor(
    learning_rate=0.01, 
    max_depth=5,  # Increase from 3 to 5
    n_estimators=500,  # Increase from 300 to 500
    subsample=0.7, 
    objective='reg:squarederror',
    colsample_bytree=0.8  # Ensure different features are used in each tree
)

xgb.fit(X_train, y_train)

print("Train R²:", xgb.score(X_train, y_train))
print("Test R²:", xgb.score(X_test, y_test))

Train R²: 0.7397935390472412
Test R²: 0.4653024673461914


In [44]:
xgb = XGBRegressor(
    learning_rate=0.01, 
    max_depth=5,  
    n_estimators=500,  
    subsample=0.7,  
    colsample_bytree=0.8,
    reg_alpha=0.1,  # L1 regularization (Lasso) to remove weak features
    reg_lambda=0.5  # L2 regularization (Ridge) to prevent overfitting
)

xgb.fit(X_train, y_train)
print("Train R²:", xgb.score(X_train, y_train))
print("Test R²:", xgb.score(X_test, y_test))

Train R²: 0.7474980354309082
Test R²: 0.4647330641746521


In [45]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500, max_depth=7, random_state=0)
rf.fit(X_train, y_train)

rf_score = rf.score(X_test, y_test)
print("Random Forest R²:", rf_score)

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest R²: 0.47136313587956347


In [46]:
xgb = XGBRegressor(
    learning_rate=0.01, 
    max_depth=5,  
    n_estimators=500,  
    subsample=0.7,  
    colsample_bytree=0.8,
    reg_alpha=1.0,  # Increase L1 regularization (default 0.0)
    reg_lambda=2.0  # Increase L2 regularization (default 1.0)
)

xgb.fit(X_train, y_train)
print("Train R²:", xgb.score(X_train, y_train))
print("Test R²:", xgb.score(X_test, y_test))

Train R²: 0.6819570064544678
Test R²: 0.47132933139801025


In [47]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

stacked = StackingRegressor(
    estimators=[('xgb', xgb), ('rf', rf)], 
    final_estimator=Ridge(alpha=1.0)  # Ridge helps with overfitting
)

stacked.fit(X_train, y_train)
print("Stacked Model R²:", stacked.score(X_test, y_test))

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\ensemble\_stacking.py:1060: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Stacked Model R²: 0.4724809611822228


In [48]:
from sklearn.feature_selection import RFE

selector = RFE(xgb, n_features_to_select=6)
selector.fit(X_train, y_train)

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

xgb.fit(X_train_selected, y_train)
print("XGBoost with Feature Selection R²:", xgb.score(X_test_selected, y_test))

XGBoost with Feature Selection R²: 0.4679340720176697


In [49]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

xgb.fit(X_train_poly, y_train)
print("XGBoost with Polynomial Features R²:", xgb.score(X_test_poly, y_test))


XGBoost with Polynomial Features R²: 0.46935176849365234


In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)

print("Accuracy on training set:", model.score(X_train_scaled, y_train))
print("Accuracy on test set:", model.score(X_test_scaled, y_test))

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on training set: 0.8190208667736758
Accuracy on test set: 0.8540145985401459


In [64]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],  
    'gamma': ['scale', 0.01, 0.1, 1],  
    'kernel': ['linear', 'rbf']  
}


grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
svc = SVC(kernel='rbf', C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'], probability=True, class_weight='balanced')

voting = VotingClassifier(estimators=[('rf', rf), ('svc', svc)], voting='soft', weights=[1, 2])
voting.fit(X_train_scaled, y_train)

print("Ensemble Model Accuracy:", voting.score(X_test_scaled, y_test))

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversio

Ensemble Model Accuracy: 0.8083941605839416


In [65]:
test_data.fillna(0, inplace=True)     
test = test_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]
predictionSVC = voting.predict(test)  
result = pd.DataFrame({'id': test_data['id'], 'prediction': predictionSVC})
result.to_csv('resultSVC.csv', index=False)

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [55]:
from sklearn.ensemble import StackingClassifier

# Create a list of models for stacking
estimators = [('rf', rf), ('svc', svc)]

stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_model.fit(X_train_scaled, y_train)

print("Stacking Model Accuracy:", stacking_model.score(X_test_scaled, y_test))

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Stacking Model Accuracy: 0.8576642335766423
